In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.datasets import make_moons
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

import IPython.display as display
from PIL import Image
import pathlib
import os

## Load the data

In [13]:
path = '/Users/test/Desktop/Master_Thesis_Flow/'

data_dir = '/Users/test/Desktop/Master_Thesis_Flow/dataset/cars_train/'
#norm = layers.experimental.preprocessing.Normalization()
#norm.adapt(data)
#normalized_data = norm(data)

data_dir = pathlib.Path(data_dir)
print(data_dir)
image_count = len(list(data_dir.glob('*.jpg')))
print(image_count)


/Users/test/Desktop/Master_Thesis_Flow/dataset/cars_train
8144


In [14]:
batch_size = 32
img_height = 128
img_width = 128

In [15]:
train_ds =tf.keras.preprocessing.image_dataset_from_directory(
  "dataset",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 8144 files belonging to 1 classes.


In [16]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "dataset",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 8144 files belonging to 1 classes.
Using 1628 files for validation.


In [17]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 128, 128, 3)
(32,)


In [18]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

normalized_data = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_data))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


## Affine Coupling Layer

In [19]:
# Creating a custom layer with keras API.
output_dim = 256
reg = 0.01


def Coupling(input_shape):
    input = keras.layers.Input(shape=input_shape)

    t_layer_1 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(input)
    t_layer_2 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_1)
    t_layer_3 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_2)
    t_layer_4 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_3)
    t_layer_5 = keras.layers.Dense(
        input_shape, activation="linear", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_4)

    s_layer_1 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(input)
    s_layer_2 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_1)
    s_layer_3 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_2)
    s_layer_4 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_3)
    s_layer_5 = keras.layers.Dense(
        input_shape, activation="tanh", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_4)

    return keras.Model(inputs=input, outputs=[s_layer_5, t_layer_5])

In [20]:
a = Coupling(2)
a.summary()

Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
dense_75 (Dense)                (None, 256)          768         input_8[0][0]                    
__________________________________________________________________________________________________
dense_70 (Dense)                (None, 256)          768         input_8[0][0]                    
__________________________________________________________________________________________________
dense_76 (Dense)                (None, 256)          65792       dense_75[0][0]                   
______________________________________________________________________________________

## Real NVP

In [23]:
class RealNVP(keras.Model):
    def __init__(self, num_coupling_layers):
        super(RealNVP, self).__init__()

        self.num_coupling_layers = num_coupling_layers

        # Distribution of the latent space.
        self.distribution = tfp.distributions.MultivariateNormalDiag(
            loc=[0.0, 0.0], scale_diag=[1.0, 1.0]
        )
        self.masks = np.array(
            [[0, 1], [1, 0]] * (num_coupling_layers // 2), dtype="float32"
        )
        self.loss_tracker = keras.metrics.Mean(name="loss")
        #Calculating values of s and t from coupling function
        self.layers_list = [Coupling(2) for i in range(num_coupling_layers)]

#     @property
#     def metrics(self):
#         """List of the model's metrics.
#         We make sure the loss tracker is listed as part of `model.metrics`
#         so that `fit()` and `evaluate()` are able to `reset()` the loss tracker
#         at the start of each epoch and at the start of an `evaluate()` call.
#         """
#         return [self.loss_tracker]

    def call(self, x, training=True):
        log_det_inv = 0
        direction = 1
        if training:
            direction = -1
        for i in range(self.num_coupling_layers)[::direction]:
            x_masked = x * self.masks[i]
            reversed_mask = 1 - self.masks[i]
            s, t = self.layers_list[i](x_masked)
            s *= reversed_mask
            t *= reversed_mask
            gate = (direction - 1) / 2
            #Equation
            x = (
                reversed_mask
                * (x * tf.exp(direction * s) + direction * t * tf.exp(gate * s))
                + x_masked
            )
            log_det_inv += gate * tf.reduce_sum(s, [1])

        return x, log_det_inv

    # Log likelihood of the normal distribution plus the log determinant of the jacobian.

    def log_loss(self, x):
        y, logdet = self(x)
        log_likelihood = self.distribution.log_prob(y) + logdet
        return -tf.reduce_mean(log_likelihood)

    def train_step(self, normalized_data):
        with tf.GradientTape() as tape:     #differentiation

            loss = self.log_loss(normalized_data)

        g = tape.gradient(loss, self.trainable_variables)  #gradient(g) = (d(loss/trainable_variables))
        self.optimizer.apply_gradients(zip(g, self.trainable_variables))
        self.loss_tracker.update_state(loss)

        return {"loss": self.loss_tracker.result()}

    def test_step(self, normalized_data):
        loss = self.log_loss(normalized_data)
        self.loss_tracker.update_state(loss)

        return {"loss": self.loss_tracker.result()}


## Training the model


In [24]:
model = RealNVP(num_coupling_layers=6)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001))

history = model.fit(
    normalized_data, epochs=30, validation_data=val_ds
)


Epoch 1/30


TypeError: in user code:

    /opt/anaconda3/envs/flow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-10-6114da8fb13f>:33 call  *
        x_masked = x * self.masks[i]
    /opt/anaconda3/envs/flow/lib/python3.8/site-packages/tensorflow/python/ops/math_ops.py:1140 binary_op_wrapper
        raise e
    /opt/anaconda3/envs/flow/lib/python3.8/site-packages/tensorflow/python/ops/math_ops.py:1124 binary_op_wrapper
        return func(x, y, name=name)
    /opt/anaconda3/envs/flow/lib/python3.8/site-packages/tensorflow/python/ops/math_ops.py:1456 _mul_dispatch
        return multiply(x, y, name=name)
    /opt/anaconda3/envs/flow/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /opt/anaconda3/envs/flow/lib/python3.8/site-packages/tensorflow/python/ops/math_ops.py:508 multiply
        return gen_math_ops.mul(x, y, name)


## Performance Evaluation 

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.legend(["train", "validation"], loc="upper right")
plt.ylabel("loss")
plt.xlabel("epoch")

# From data to latent space.
z, _ = model(normalized_data)

# From latent space to data.
samples = model.distribution.sample(3000)
x, _ = model.predict(samples)

f, axes = plt.subplots(2, 2)
f.set_size_inches(20, 15)

axes[0, 0].scatter(normalized_data[:, 0], normalized_data[:, 1], color="r")
axes[0, 0].set(title="Inference data space X", xlabel="x", ylabel="y")
axes[0, 1].scatter(z[:, 0], z[:, 1], color="r")
axes[0, 1].set(title="Inference latent space Z", xlabel="x", ylabel="y")
axes[0, 1].set_xlim([-3.5, 4])
axes[0, 1].set_ylim([-4, 4])
axes[1, 0].scatter(samples[:, 0], samples[:, 1], color="g")
axes[1, 0].set(title="Generated latent space Z", xlabel="x", ylabel="y")
axes[1, 1].scatter(x[:, 0], x[:, 1], color="g")
axes[1, 1].set(title="Generated data space X", label="x", ylabel="y")
axes[1, 1].set_xlim([-2, 2])
axes[1, 1].set_ylim([-2, 2])